<a href="https://colab.research.google.com/github/Satyendra0207/Spark_collab/blob/main/collb_Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 54.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=15e77524061c4e11165957925805b4785a3048de3f665b086c66eed03497083f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=6f96faf1f1b5dc3720951c0f0d0b054f721aabb5dc7726bb4ccc9f41d26e3a0b
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import wget
wget.download("https://files.grouplens.org/datasets/movielens/ml-25m.zip",'ml_25.zip')

'ml_25.zip'

In [ ]:
from zipfile import ZipFile

In [ ]:
#extracting zip file
with ZipFile("ml_25.zip",'r') as zip:
  zip.printdir()
  zip.extractall()

File Name                                             Modified             Size
ml-25m/                                        2019-11-21 16:41:38            0
ml-25m/tags.csv                                2019-11-21 15:55:50     38810332
ml-25m/links.csv                               2019-11-21 16:40:20      1368578
ml-25m/README.txt                              2019-11-21 16:41:38        10460
ml-25m/ratings.csv                             2019-11-21 16:26:42    678260987
ml-25m/genome-tags.csv                         2019-11-21 16:35:38        18103
ml-25m/genome-scores.csv                       2019-11-21 16:35:38    435164157
ml-25m/movies.csv                              2019-11-21 16:32:10      3038099


In [ ]:
from pyspark.sql import SparkSession
session=SparkSession.builder.appName("recomm").getOrCreate()

In [ ]:
data=session.read.csv("ml-25m/ratings.csv",header=True,inferSchema=True)

In [ ]:
data.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
+------+-------+------+----------+
only showing top 5 rows



In [ ]:
data.count()

25000095

In [ ]:
#
from pyspark.sql.functions import countDistinct
data.select(countDistinct("userId")).show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                162541|
+----------------------+



In [ ]:
data.select(countDistinct("movieId")).show()

+-----------------------+
|count(DISTINCT movieId)|
+-----------------------+
|                  59047|
+-----------------------+



In [ ]:
from pyspark.sql.functions import max,min,mean


In [ ]:
data.select(min("rating"),max("rating")).show()

+-----------+-----------+
|min(rating)|max(rating)|
+-----------+-----------+
|        0.5|        5.0|
+-----------+-----------+



In [ ]:
training,test=data.randomSplit([0.75,0.25])
print(training.count())
print(test.count())

18750434
6249661


In [ ]:
from pyspark.ml.recommendation import ALS
model=ALS(userCol="userId",itemCol="movieId",ratingCol="rating")
newmodel=model.fit(training)
result=newmodel.transform(test)
result.show(8)

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|     1|   1088|   4.0|1147868495| 2.6291485|
|     1|   1217|   3.5|1147878326| 4.1758556|
|     1|   1250|   4.0|1147868414| 3.8516183|
|     1|   2573|   4.0|1147878923| 3.2164664|
|     1|   4308|   3.0|1147868534| 3.5695515|
|     1|   4422|   3.0|1147869048|  4.025502|
|     1|   4703|   4.0|1147869223| 3.6291504|
|     1|   5767|   5.0|1147878729| 4.2268286|
+------+-------+------+----------+----------+
only showing top 8 rows



In [ ]:
#in a test dataset the rating column is not considered and the value is predicted on the basis 
#of userId anf movieId
result.filter(result['movieId']==1217).orderBy('prediction',ascending=True).show(5)

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|154776|   1217|   0.5|1448674961| 0.8510009|
| 13838|   1217|   1.0|1448155316| 1.1172059|
| 26097|   1217|   0.5|1474499745| 1.1493871|
| 57548|   1217|   4.0|1452183575| 1.3717135|
| 38084|   1217|   1.0|1467438624| 1.5562804|
+------+-------+------+----------+----------+
only showing top 5 rows



In [ ]:
result.filter(result['userId']==4377).orderBy('prediction',ascending=True).show(5)

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|  4377|    169|   1.0|974780209| 1.8304178|
|  4377|    519|   2.0|974801284| 1.9928397|
|  4377|    473|   1.0|974782219| 2.0183134|
|  4377|   1985|   2.0|974780566| 2.2105515|
|  4377|   3031|   2.0|974803930| 2.2449143|
+------+-------+------+---------+----------+
only showing top 5 rows

